# Competitive Interaction on SummEval

## Imports

In [19]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager
import re
from tqdm import tqdm
import numpy as np

## Data

In [20]:
SummEval = load_dataset("mteb/summeval")

df = pd.DataFrame(SummEval["test"])[["text", "machine_summaries", "relevance", "coherence", "fluency", "consistency"]]

problematic_indices = [5, 7, 8, 9, 10, 11, 18, 20, 26, 27, 33, 34, 39, 46, 61, 64, 68, 73, 75, 79, 85, 86, 88, 92, 96, 99]
df_filtered = df.drop(index=problematic_indices).reset_index(drop=True)

df_exploded = df_filtered.explode(["machine_summaries", "relevance", "coherence", "fluency", "consistency"]).reset_index(drop=True)

df_sampled = df_exploded.iloc[:1200].sample(n=100, random_state=42).reset_index(drop=True)

columns_to_round = ["relevance", "coherence", "fluency", "consistency"]

df_sampled[columns_to_round] = df_sampled[columns_to_round].map(lambda x: int(np.ceil(x) if x % 1 > 0.5 else np.floor(x)))

df_final = df_sampled

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               100 non-null    object
 1   machine_summaries  100 non-null    object
 2   relevance          100 non-null    int64 
 3   coherence          100 non-null    int64 
 4   fluency            100 non-null    int64 
 5   consistency        100 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 4.8+ KB
None
                                                text  \
0  Boss Nigel Pearson has urged Leicester to keep...   

                                   machine_summaries  relevance  coherence  \
0  jamie vardy scored an injury-time winner again...          2          2   

   fluency  consistency  
0        3            2  


## Config

In [21]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

llm_config={"config_list": config_list}

## System Design

In [22]:
agent_1_system_message =f"""
You are a super critical Debate-Agent.
Your task is to find flaws, inconsistencies, and oversights, and present them with clarity and force.
Argue convincingly and with clear reasoning why the summary is bad to persuade the Decision-Agent.
You are only allowed to use negative arguments.

In this task you will evaluate the quality of a summary written for a news article.
To correctly solve this task, follow these steps:

    1. Carefully read the news article, be aware of the information it contains.
    2. Read the proposed summary.
    3. Rate the summary with integer values on a scale from 1 (worst) to 5 (best) by its relevance, consistency, fluency, and coherence.

Definitions:
    Relevance:
        - The rating measures how well the summary captures the key points of the article.
        - Consider whether all and only the important aspects are contained in the summary.
    Consistency:
        - The rating measures whether the facts in the summary are consistent with the facts in the original article.
        - Consider whether the summary does reproduce all facts accurately and does not make up untrue information.
    Fluency:
        - This rating measures the quality of individual sentences, are they well-written and grammatically correct.
        - Consider the quality of individual sentences.
    Coherence:
        - The rating measures the quality of all sentences collectively, to the fit together and sound naturally.
        - Consider the quality of the summary as a whole

Use a structured format and about 200 words to present your arguments.
Always begin your output with: "As an independent evaluator I think ..."
Always end your output with a JSON object with the following format:{{"relevance": score, "coherence": score, "fluency": score, "consistency": score}} 
"""


agent_2_system_message =f"""
You are a super tolerant Debate-Agent.
Your task is to recognize strengths, positive choices, and thoughtful decisions, and present them with clarity and force.
Argue convincingly and with clear reasoning why the summary is good to persuade the Decision-Agent.
You are only allowed to use positive arguments.

In this task you will evaluate the quality of a summary written for a news article.
To correctly solve this task, follow these steps:

    1. Carefully read the news article, be aware of the information it contains.
    2. Read the proposed summary.
    3. Rate the summary with integer values on a scale from 1 (worst) to 5 (best) by its relevance, consistency, fluency, and coherence.

Definitions:
    Relevance:
        - The rating measures how well the summary captures the key points of the article.
        - Consider whether all and only the important aspects are contained in the summary.
    Consistency:
        - The rating measures whether the facts in the summary are consistent with the facts in the original article.
        - Consider whether the summary does reproduce all facts accurately and does not make up untrue information.
    Fluency:
        - This rating measures the quality of individual sentences, are they well-written and grammatically correct.
        - Consider the quality of individual sentences.
    Coherence:
        - The rating measures the quality of all sentences collectively, to the fit together and sound naturally.
        - Consider the quality of the summary as a whole

Use a structured format and about 200 words to present your arguments.
Always begin your output with: "As an independent evaluator I think ..."
Always end your output with a JSON object with the following format:{{"relevance": score, "coherence": score, "fluency": score, "consistency": score}} 
"""

agent_3_system_message ="""
You are the final Decision-Agent in open debate setting.
Your task is to evaluate the quality of a summary written for a news article and make a final judgment.
You must carefully review the evaluators' previous analyses and refer to them in your explanation.

In this task you will evaluate the quality of a summary written for a news article.
To correctly solve this task, follow these steps:

    1. Carefully read the news article, be aware of the information it contains.
    2. Read the proposed summary.
    3. Rate the summary with integer values on a scale from 1 (worst) to 5 (best) by its relevance, consistency, fluency, and coherence.

Definitions:
    Relevance:
        - The rating measures how well the summary captures the key points of the article.
        - Consider whether all and only the important aspects are contained in the summary.
    Consistency:
        - The rating measures whether the facts in the summary are consistent with the facts in the original article.
        - Consider whether the summary does reproduce all facts accurately and does not make up untrue information.
    Fluency:
        - This rating measures the quality of individual sentences, are they well-written and grammatically correct.
        - Consider the quality of individual sentences.
    Coherence:
        - The rating measures the quality of all sentences collectively, to the fit together and sound naturally.
        - Consider the quality of the summary as a whole

Give an explanation for your final conclusion using about 200 words.
Always begin your output with: "As an final Decision-Agent, after carefully reviewing the evaluators' previous analyses, I think ..."
Always end your output with a JSON object with the following format:{{"relevance": score, "coherence": score, "fluency": score, "consistency": score}} 
"""

In [23]:
agent_1 = ConversableAgent(
    name="Critical-Debate-Agent",
    llm_config=llm_config,
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)

agent_2 = ConversableAgent(
    name="Tolerant-Debate-Agent",
    llm_config=llm_config,
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)

agent_3 = ConversableAgent(
    name="Decision-Agent",
    llm_config=llm_config,
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

group_chat = GroupChat(
    agents=[agent_1, agent_2, agent_3],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

## Evaluation

In [24]:
def evaluate(text, machine_summaries, relevance, coherence, fluency, consistency):
    message = f""" 
    Article: {text}

    Summary: {machine_summaries}
    """

    chat_results = agent_3.initiate_chat(
        group_chat_manager,
        message=message,
        summary_method="last_msg",
    )

    result_str = str(chat_results.chat_history[-1]["content"])
    print(result_str)

    pattern = r'"relevance"\s*:\s*(\d+)'
    relevance_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"coherence"\s*:\s*(\d+)'
    coherence_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"fluency"\s*:\s*(\d+)'
    fluency_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"consistency"\s*:\s*(\d+)'
    consistency_score = int(re.search(pattern, result_str).group(1))

    relevance_deviation = abs(relevance - relevance_score) 
    coherence_deviation = abs(coherence - coherence_score) 
    fluency_deviation = abs(fluency - fluency_score) 
    consistency_deviation = abs(consistency - consistency_score) 

    return {
        "relevance": {
            "ground_truth": relevance,
            "system_decision": relevance_score,
            "deviation": relevance_deviation
        },
        "coherence": {
            "ground_truth": coherence,
            "system_decision": coherence_score,
            "deviation": coherence_deviation
        },
        "fluency": {
            "ground_truth": fluency,
            "system_decision": fluency_score,
            "deviation": fluency_deviation
        },
        "consistency": {
            "ground_truth": consistency,
            "system_decision": consistency_score,
            "deviation": consistency_deviation
        }
    }

In [25]:
num_rows = 3

results = []

df_subset = df_final.head(num_rows)

for _, row in tqdm(df_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        text=row["text"],
        machine_summaries=row["machine_summaries"],
        relevance=row["relevance"],
        coherence=row["coherence"],
        fluency=row["fluency"],
        consistency=row["consistency"]
    )
    results.append(result)

results_df = pd.DataFrame(results)
results_df.to_csv('Results/competitive.csv', index=False)

Progress:   0%|          | 0/3 [00:00<?, ?it/s]

Decision-Agent (to chat_manager):

 
    Article: Boss Nigel Pearson has urged Leicester to keep their cool and ignore their relegation rivals. The Foxes host Swansea on Saturday just three points from safety in the Barclays Premier League after back-to-back wins. Last week's 3-2 win at West Brom handed them a survival lifeline, although they remain bottom of the table. Jamie Vardy scored an injury-time winner against West Bromwich Albion on Saturday to improve his side's slim chance of Premier League survival Vardy celebrates in front of the travelling away fans after hitting the winner against West Brom But after their mini-revival, Pearson wants his side to remain focused on their own jobs. 'I'm very wary of people flipping the emphasis,' he said. 'Our future is in our own hands and if we go into the last game with that we have given ourselves a realistic chance. 'We need to make sure our own run-in is what we want it to be. Leicester manager Nigel Pearson has urged his players to f

Progress:  33%|███▎      | 1/3 [00:05<00:11,  5.66s/it]

As a final Decision-Agent, after carefully reviewing the evaluators' previous analyses, I think the proposed summary has significant shortcomings that hinder its effectiveness. 

The relevance of the summary is notably low, as it fails to encapsulate the article's main themes, particularly the importance of Leicester's focus on their own performance amidst relegation concerns. While it mentions Jamie Vardy's goal, it does not adequately convey the context of Leicester being at the bottom of the table or the implications of their recent wins. The phrase "swansea host swansea" is a critical error that misrepresents the situation and detracts from the summary's relevance.

In terms of consistency, the summary inaccurately states that "west brom are bottom of the table," which contradicts the article's assertion that Leicester is in that position. This inconsistency undermines the summary's credibility.

Fluency is also an issue, as the summary lacks grammatical correctness and flows poorl

Progress:  67%|██████▋   | 2/3 [00:08<00:04,  4.01s/it]

As a final Decision-Agent, after carefully reviewing the evaluators' previous analyses, I think the proposed summary has both strengths and weaknesses that need to be addressed. 

The summary does capture the central theme of Carlos Tevez wanting to leave Juventus to return to Boca Juniors, which is a key point of the article. However, it lacks depth by omitting important context, such as Tevez's current success at Juventus, including his potential second Serie A title and his status as the league's top scorer. This omission affects the relevance of the summary, as it does not provide a complete picture of Tevez's situation.

In terms of consistency, while the summary correctly states that Boca is close to completing a deal, it simplifies the situation by not emphasizing the necessity for Tevez to terminate his contract first, which is a crucial detail. The fluency of the summary is adequate, with grammatically correct sentences, but the overall structure feels somewhat simplistic and 

Progress: 100%|██████████| 3/3 [00:12<00:00,  4.02s/it]

As a final Decision-Agent, after carefully reviewing the evaluators' previous analyses, I think the proposed summary has both strengths and weaknesses that need to be addressed. 

The summary does capture some key points of the article, such as the auction of the dress worn by Vivien Leigh and its sale price of $137,000. However, it fails to include significant details about the dress's original color and its fading over time, which are important for understanding its historical context. Additionally, the backstory of James Tumblin, who saved the dress from being discarded, is omitted, which detracts from the relevance of the summary.

In terms of consistency, while the summary accurately mentions the auction details, it lacks clarity regarding the dress being part of a larger collection. The fluency of the summary is hindered by poor punctuation and capitalization, making it less readable. Lastly, the coherence is weak, as the sentences do not flow logically, and there is redundancy i